In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

/home/surya/DL/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
import spacy
nlp = spacy.load('en')

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

In [4]:
max_len = max([len(i.split()) for i in train.text.values])
max_len

861

In [3]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [6]:
lbl_encoder = preprocessing.LabelEncoder()
y = lbl_encoder.fit_transform(train.author.values)

In [7]:
x_train,x_valid,y_train,y_valid = train_test_split(train.text.values,y,
                                                       test_size=0.5,shuffle=True)

In [12]:
x_train_fltr = [[token.text for token in nlp(doc) if not token.is_stop 
                 and token.text not in ['.',',',':']]for doc in x_train]
x_valid_fltr = [[token.text for token in nlp(doc) if not token.is_stop
                 and token.text not in ['.',',',':']]for doc in x_valid]


In [ ]:
x_train_fltr

In [36]:
x_train_fltr_join = [' '.join(doc) for doc in x_train_fltr]
x_valid_fltr_join = [' '.join(doc) for doc in x_valid_fltr]

In [37]:
x_train_fltr_join

['You careful thing downright improviso air',
 'In manner I fancied I distinguish precise moment fairly disengaged lid I determine removed altogether deposited lower berth room ; point I knew example certain slight taps lid striking wooden edges berth endeavored lay gently room floor',
 'So White Ship sailed past walls Thalarion followed days southward flying bird glossy plumage matched sky appeared',
 'God I dare tell I saw',
 'Oh beloved father Indeed miserable words truly I forgive entirely possess heart I endeavoured rainbow gleams cataract D soften thy tremendous sorrows',
 'At I beheld seething blur luminosity ; shapes infinitely distant began detach confusion I saw Juan Romero ?',
 'I wished shake thought feeling I learned means overcome sensation pain death state I feared understand',
 'To accomplish great desideratum ærial navigation generally supposed exceedingly complicated application unusually profound principle dynamics',
 'It Greenwich Greenwich roof row houses lovely gr

In [38]:
xtrain = np.array(x_train_fltr_join)
xvalid = np.array(x_valid_fltr_join)

In [39]:
xtrain.shape,xvalid.shape,y_train.shape,y_valid.shape

((9789,), (9790,), (9789,), (9790,))

In [8]:
from spacy.vectors import Vectors
import spacy

nlp =  spacy.load('en')
#nlp.vocab.vectors.from_glove('data/glove.840B.300d.txt')

In [ ]:
xtrain_token=[]
for i in x_train:
    doc = nlp(i)
    xtrain_token.append([token.text for token in doc ])

In [ ]:
xvalid_token=[]
for i in x_valid:
    doc = nlp(i)
    xvalid_token.append([token.text for token in doc ])


In [24]:
loc = "data/"
import bcolz
import pickle
import sys   
import pdb
import os
from tensorflow.contrib.learn.python import preprocessing

In [40]:
p = preprocessing.text.VocabularyProcessor(max_document_length=500)
xtrain_ids = np.array(list(p.fit_transform(x_train_fltr_join)))
xvalid_ids = np.array(list(p.fit_transform(x_valid_fltr_join)))

In [41]:
xtrain_ids.shape,xvalid_ids.shape

((9789, 500), (9790, 500))

In [43]:
xtrain_ids_v[np.where(xtrain_ids_v > 21000)].size

0

In [8]:
vocab_size = 21000

In [42]:
xtrain_ids_v = np.array([[i if i<vocab_size else vocab_size-1 for i in s] for s in xtrain_ids])
xvalid_ids_v = np.array([[i if i<vocab_size else vocab_size-1 for i in s] for s in xvalid_ids])

In [19]:
vocab_size = 10000
seq_len = 500

### Writing the input data to a pickle object

In [4]:
import pickle

In [ ]:
file1 = open("predict_author_input.pkl","wb")

pickle.dump([xtrain_ids_v,xvalid_ids_v,ytrain_enc,yvalid_enc],file1)

In [5]:
xt,xv,yt,yv = pickle.load(open("predict_author_input.pkl","rb"))

## Embedding matrix

In [ ]:
from spacy.vectors import  Vectors


## CNN/LSTM Network for classifying the Author

In [45]:

# we need to binarize the labels for the neural net
ytrain_enc = np_utils.to_categorical(y_train)
yvalid_enc = np_utils.to_categorical(y_valid)

In [18]:
from sklearn import preprocessing as skp
scl = skp.StandardScaler()
xtrain_ids_scl = scl.fit_transform(xtrain_ids_v)
xvalid_ids_scl = scl.transform(xvalid_ids_v)

/home/surya/DL/lib/python3.5/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [6]:
from keras.layers import  Conv1D

In [61]:
model = Sequential([
    Embedding(vocab_size, 100, input_length=seq_len),
    Dropout(0.25),
    Conv1D(padding="same", kernel_size=3, filters=64, activation="relu"),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    #Dropout(0.2),
    Dense(3, activation='softmax')])

In [62]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')

In [63]:
model.optimizer.lr=0.1
model.fit(xt,yt,batch_size=128,epochs=5,
          validation_data=(xv,yv))
model.save_weights('model1_weights.h5')

Train on 9789 samples, validate on 9790 samples
Epoch 1/5
9789/9789 [==============================] - 29s 3ms/step - loss: 10.8997 - acc: 0.3159 - val_loss: 11.2794 - val_acc: 0.3002
Epoch 2/5
9789/9789 [==============================] - 28s 3ms/step - loss: 11.0056 - acc: 0.3172 - val_loss: 11.2794 - val_acc: 0.3002
Epoch 3/5
9789/9789 [==============================] - 27s 3ms/step - loss: 11.0056 - acc: 0.3172 - val_loss: 11.2794 - val_acc: 0.3002
Epoch 4/5
8576/9789 [=========================>....] - ETA: 3s - loss: 10.9929 - acc: 0.3180

KeyboardInterrupt: 

In [37]:
model_bn = Sequential([
    Embedding(vocab_size, 300, input_length=seq_len,  
             trainable=True),
    BatchNormalization(),
    Dropout(0.25),
    Conv1D(padding="same", kernel_size=5, filters=64, activation="relu"),
    BatchNormalization(),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax')])
model_bn.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [38]:
model_bn.fit(xtrain_ids_v,ytrain_enc,batch_size=256,epochs=10,
          validation_data=(xvalid_ids_v,yvalid_enc))

Train on 9789 samples, validate on 9790 samples
Epoch 1/10
9789/9789 [==============================] - 88s 9ms/step - loss: 1.4586 - acc: 0.3753 - val_loss: 1.1250 - val_acc: 0.4101
Epoch 2/10
9789/9789 [==============================] - 87s 9ms/step - loss: 1.0405 - acc: 0.5275 - val_loss: 1.2078 - val_acc: 0.4101
Epoch 3/10
9789/9789 [==============================] - 87s 9ms/step - loss: 0.5358 - acc: 0.7979 - val_loss: 1.2457 - val_acc: 0.4101
Epoch 4/10
9789/9789 [==============================] - 87s 9ms/step - loss: 0.2053 - acc: 0.9331 - val_loss: 1.2488 - val_acc: 0.4101
Epoch 5/10
9789/9789 [==============================] - 87s 9ms/step - loss: 0.1017 - acc: 0.9710 - val_loss: 1.2370 - val_acc: 0.4101
Epoch 6/10
9789/9789 [==============================] - 87s 9ms/step - loss: 0.0542 - acc: 0.9863 - val_loss: 1.3065 - val_acc: 0.4101
Epoch 7/10
9789/9789 [==============================] - 88s 9ms/step - loss: 0.0317 - acc: 0.9941 - val_loss: 1.3343 - val_acc: 0.4101
Epoch 8

In [16]:
model_l = Sequential()
model_l.add(Embedding(vocab_size,
                     300,
                     input_length=seq_len,
                     trainable=True))
model_l.add(SpatialDropout1D(0.3))
model_l.add(GRU(300, dropout=0.3, recurrent_dropout=0.3, return_sequences=True))
model_l.add(GRU(300, dropout=0.3, recurrent_dropout=0.3))

model_l.add(Dense(1024, activation='relu'))
model_l.add(Dropout(0.8))

model_l.add(Dense(1024, activation='relu'))
model_l.add(Dropout(0.8))

model_l.add(Dense(3))
model_l.add(Activation('softmax'))
model_l.compile(loss='categorical_crossentropy', optimizer='adam')

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model_l.fit(xtrain_ids_v, y=ytrain_enc, batch_size=512, epochs=10, 
          verbose=1, validation_data=(xvalid_ids_v, yvalid_enc), callbacks=[earlystop])

Train on 9789 samples, validate on 9790 samples
Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[512,500,300]
	 [[Node: gru_4/zeros_like = ZerosLike[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](gru_3/transpose_1)]]
	 [[Node: loss_1/mul/_179 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3975_loss_1/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'gru_4/zeros_like', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/surya/DL/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/surya/DL/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
  

In [50]:
# A simple bidirectional LSTM with glove embeddings and two dense layers
model2 = Sequential()
model2.add(Embedding(vocab_size,
                     64,
                     input_length=seq_len,
                     trainable=True))
model2.add(SpatialDropout1D(0.3))
model2.add(Bidirectional(LSTM(64, dropout=0.3, recurrent_dropout=0.3)))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(1024, activation='relu'))
model2.add(Dropout(0.8))

model2.add(Dense(3))
model2.add(Activation('softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

# Fit the model with early stopping callback
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto')
model2.fit(xt, y=yt, batch_size=512, epochs=10, 
          verbose=1, validation_data=(xv, yv), callbacks=[earlystop])

Train on 9789 samples, validate on 9790 samples
Epoch 1/10
9789/9789 [==============================] - 164s 17ms/step - loss: 1.0879 - acc: 0.3988 - val_loss: 1.0806 - val_acc: 0.4040
Epoch 2/10
9789/9789 [==============================] - 161s 16ms/step - loss: 1.0662 - acc: 0.4270 - val_loss: 1.0658 - val_acc: 0.4581
Epoch 3/10
9789/9789 [==============================] - 162s 17ms/step - loss: 0.9704 - acc: 0.5429 - val_loss: 0.9752 - val_acc: 0.5461
Epoch 4/10
9789/9789 [==============================] - 175s 18ms/step - loss: 0.8510 - acc: 0.6094 - val_loss: 0.9600 - val_acc: 0.5727
Epoch 5/10
9789/9789 [==============================] - 225s 23ms/step - loss: 0.7768 - acc: 0.6471 - val_loss: 0.9077 - val_acc: 0.6064
Epoch 6/10
9789/9789 [==============================] - 179s 18ms/step - loss: 0.6984 - acc: 0.7098 - val_loss: 0.8735 - val_acc: 0.6413
Epoch 7/10
9789/9789 [==============================] - 160s 16ms/step - loss: 0.6250 - acc: 0.7596 - val_loss: 0.8218 - val_acc: 

In [51]:
model2.optimizer.lr=0.1
model2.fit(xt, y=yt, batch_size=512, epochs=3, 
          verbose=1, validation_data=(xv, yv), callbacks=[earlystop])

Train on 9789 samples, validate on 9790 samples
Epoch 1/3
9789/9789 [==============================] - 160s 16ms/step - loss: 0.4421 - acc: 0.8329 - val_loss: 0.7880 - val_acc: 0.6962
Epoch 2/3
9789/9789 [==============================] - 156s 16ms/step - loss: 0.4174 - acc: 0.8403 - val_loss: 0.7993 - val_acc: 0.7044
Epoch 3/3
9789/9789 [==============================] - 160s 16ms/step - loss: 0.3863 - acc: 0.8519 - val_loss: 0.8664 - val_acc: 0.6799
